In [4]:
### Метод непосредственного развёртывания для нахождения собственных значений и собственных векторов

import sympy as sp


def char_poly_direct(A):
    """
    Вычисляет характеристический многочлен матрицы A методом непосредственного развертывания детерминанта.

    Параметры:
    A : list[list[число]]
        Квадратная матрица размера n*n в виде вложенных списков.

    Возвращает:
    sp.Poly
        Объект символьного многочлена по переменной lambda.
    """
    n = len(A)
    lam = sp.symbols('lam')
    ## построение символической матрицы B = A - lam*I
    B = [[A[i][j] - lam if i == j else A[i][j] for j in range(n)] for i in range(n)]

    def det(M):
        ### рекурсивное вычисление детерминанта развертыванием по первой строке
        size = len(M)
        if size == 1:
            return M[0][0]
        total = 0
        for j in range(size):
            ### формируем минор, исключая строку 0 и столбец j
            minor = [row[:j] + row[j+1:] for row in M[1:]]
            total += (-1)**j * M[0][j] * det(minor)
        return total

    ### вычисление и упрощение детерминанта
    poly_expr = sp.expand(det(B))
    return sp.Poly(poly_expr, lam)


def eigenvalues_and_vectors_direct(A):
    """
    Находит собственные значения и собственные векторы матрицы A,
    используя характеристический многочлен, найденный методом непосредственного развертывания.

    Параметры:
    A : list[list[число]]
        Квадратная матрица размера n*n.

    Возвращает:
    tuple:
        eigenvalues : list[число]
            Список собственных значений lambda_1, lambda_2, ..., lambda_n.
        eigenvectors : dict
            Словарь, где ключ — собственное значение, значение — список собственных векторов (sp.Matrix).
    """
    lam = sp.symbols('lam')
    ### получаем характеристический многочлен
    poly = char_poly_direct(A)
    ### решаем уравнение P(lambda) = 0
    eigenvals = sp.solve(poly.as_expr(), lam)
    ### представим A в виде sp.Matrix для поиска нулевого пространства
    M = sp.Matrix(A)
    eigenvects = {}
    for val in eigenvals:
        nullsp = (M - val * sp.eye(M.shape[0])).nullspace()
        eigenvects[val] = nullsp
    return eigenvals, eigenvects


A = [[3, -2], [-4, 1]]
eigenvals, eigenvects = eigenvalues_and_vectors_direct(A)
print("Собственные значения:", eigenvals)
for val, vecs in eigenvects.items():
    print(f"Для lambda = {val}:")
    for v in vecs:
        print(v)


Собственные значения: [-1, 5]
Для lambda = -1:
Matrix([[1/2], [1]])
Для lambda = 5:
Matrix([[-1], [1]])


In [3]:
### Метод итераций (степенной метод)

import numpy as np


def power_method(A, x0=None, tol=1e-6, max_iter=1000):
    """
    Реализует степенной метод для приближённого поиска наибольшего по модулю собственного значения
    и соответствующего собственного вектора матрицы A.

    Параметры:
    A : array_like
        Квадратная матрица размера n*n.
    x0 : array_like, optional
        Начальный вектор (длина n). Если не задан, используется вектор единиц.
    tol : float, optional
        Точность критерия останова по изменению собственного значения.
    max_iter : int, optional
        Максимальное число итераций.

    Возвращает:
    lambda_max : float
        Приближённое собственное значение наибольшего по модулю.
    v : ndarray
        Нормированный собственный вектор (норма = 1).
    num_iter : int
        Фактическое число выполненных итераций.

    Пример:
    >>> A = np.array([[2, 1], [1, 3]], dtype=float)
    >>> lambda_max, v, it = power_method(A)
    >>> print(lambda_max, v, it)
    """
    A = np.array(A, dtype=float)
    n, m = A.shape
    if n != m:
        raise ValueError("Матрица А должна быть квадратной (m == n)")

    ### инициализация вектора
    if x0 is None:
        x = np.ones(n)
    else:
        x = np.array(x0, dtype=float)
    ### нормировка начального вектора
    x = x / np.linalg.norm(x)

    lambda_old = 0.0
    for k in range(1, max_iter + 1):
        y = A.dot(x)
        ### новая приближённая собственная оценка 
        lambda_new = np.dot(x, y)
        ### нормировка y для следующей итерации
        x = y / np.linalg.norm(y)

        ### проверка сходимости
        if abs(lambda_new - lambda_old) < tol:
            break
        lambda_old = lambda_new

    return lambda_new, x, k


A = np.array([[4, 1], [2, 3]], dtype=float)
eigval, eigvec, iterations = power_method(A, tol=1e-8, max_iter=1000)
print(f"Приближённое собственное значение: {eigval}")
print(f"Собственный вектор: {eigvec}")
print(f"Итераций выполнено: {iterations}")


Приближённое собственное значение: 4.999999999999999
Собственный вектор: [0.70710678 0.70710678]
Итераций выполнено: 2
